# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Classification-based-glottal-closure-instant-detection" data-toc-modified-id="Classification-based-glottal-closure-instant-detection-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Classification-based glottal closure instant detection</a></div><div class="lev2 toc-item"><a href="#Training-the-classifier" data-toc-modified-id="Training-the-classifier-11"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Training the classifier</a></div><div class="lev2 toc-item"><a href="#Evaluating-the-classifier" data-toc-modified-id="Evaluating-the-classifier-12"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Evaluating the classifier</a></div><div class="lev3 toc-item"><a href="#Evaluation-on-the-CMU-data" data-toc-modified-id="Evaluation-on-the-CMU-data-121"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>Evaluation on the CMU data</a></div><div class="lev4 toc-item"><a href="#ARCTIC-BDL-voice" data-toc-modified-id="ARCTIC-BDL-voice-1211"><span class="toc-item-num">1.2.1.1&nbsp;&nbsp;</span>ARCTIC BDL voice</a></div><div class="lev4 toc-item"><a href="#ARCTIC-SLT-voice" data-toc-modified-id="ARCTIC-SLT-voice-1212"><span class="toc-item-num">1.2.1.2&nbsp;&nbsp;</span>ARCTIC SLT voice</a></div><div class="lev4 toc-item"><a href="#CSTR-US-KED-Timit" data-toc-modified-id="CSTR-US-KED-Timit-1213"><span class="toc-item-num">1.2.1.3&nbsp;&nbsp;</span>CSTR US KED Timit</a></div>

# Classification-based glottal closure instant detection

This is an example of a Python code to train and test a classifier used to detect glottal closure instants (GCIs). See the [corresponding paper](paper/matousek_is2017_submit.pdf) for more details.

[Scikit-learn toolkit](http://scikit-learn.org/stable/) is used to train and evaluate the classifier.

Prerequisities:
- [Python](https://www.python.org/) (version 2.7.13 used in this example)
- [Numpy](http://www.numpy.org/) (1.12.1)
- [Scipy](https://www.scipy.org/) (0.19.0)
- [Scikit-learn](http://scikit-learn.org/stable/) (0.18.1)
- [Pandas](http://pandas.pydata.org/) (0.19.2)

## Training the classifier

For classification in this example, we use an [extremely randomized trees](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.ExtraTreesClassifier.html) classifier (ERT) which was found to achieve the best performance in our experiments.

Firstly, we define a classification pipeline that consists of a [feature scaler](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) and the classifier.

In [3]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# Init classification pipeline with standard scaler and
# Extremely randomized tress classifier (with default parameters)
pipe = Pipeline([('fscale', StandardScaler()),
                ('classif', ExtraTreesClassifier(criterion='entropy', random_state=42))])

Then, training data and its targets stored in a Numpy format is loaded. Peak-based features that describe properties of the current (to-be-classified) peak in a speech waveform and of the 6 neighboring peaks (3 prior and 3 susequent), resulting in a total number of 32 features for each peak candidate, were used (see the [corresponding paper](paper/matousek_is2017_submit.pdf)). Hand-crafted GCIs were used as targets.

In [20]:
import numpy as np
# Read training data and targets
X = np.load('data/uwb/X_train_p3.npy')
y = np.load('data/uwb/y_train.npy')
print('# examples: {}'.format(X.shape[0]))
print('# features: {}'.format(X.shape[1]))

# examples: 66256
# features: 32


Before the training, a grid of relevant parameters is defined.

In [5]:
import scipy.stats as st
# Parameter grid for classification
grid = {'classif__n_estimators': st.randint(100, 251),
        'classif__max_features': st.uniform(0.5, 0.5),
        'classif__min_samples_split': st.randint(2, 6),
        'classif__min_samples_leaf': st.randint(1, 4)}

A [randomized grid search with cross-validation](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html) is then defined to search for the best classifier (hyper)parameters using a 3-fold cross-validation scheme ($cv=3$). The number of parameter settings that are sampled is set to $n\_iter=10$.

In [6]:
from sklearn.model_selection import RandomizedSearchCV
# Define the grid search and cross-validation
gs = RandomizedSearchCV(pipe, grid, n_iter=10, scoring='f1', cv=3, random_state=42, n_jobs=1)

Note that the parameters of the randomized grid search were set here to enable a faster training. In the [corresponding paper](paper/matousek_is2017_submit.pdf), $n\_iter$ was set to 20 and $cv$ was set to 10. Of course, an [exhaustive search](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) can be also employed instead.

The grid search object is then trained.

In [7]:
import pickle
# Train the grid 
gs.fit(X, y)
print('Best parameters found:')
print('# estimators     : {}'.format(gs.best_estimator_.named_steps['classif'].get_params()['n_estimators']))
print('max features     : {:.2%}'.format(gs.best_estimator_.named_steps['classif'].get_params()['max_features']))
print('min samples split: {}'.format(gs.best_estimator_.named_steps['classif'].get_params()['min_samples_split']))
print('min samples leaf : {}'.format(gs.best_estimator_.named_steps['classif'].get_params()['min_samples_leaf']))


Best parameters found:
# estimators     : 229
max features     : 98.50%
min samples split: 3
min samples leaf : 2


Summary of results for each parameter setting and each cross-validation fold could be stored to a CSV file

In [11]:
from pandas import DataFrame, ExcelWriter
# Import cross-validation results
df = DataFrame(gs.cv_results_)
df.to_csv('csv_results.txt')

or to an excel file

In [ ]:
writer = ExcelWriter('cv_results.xlsx')
df.to_excel(writer)
writer.save()

## Evaluating the classifier

The trained classifier is evaluated on the UWB test dataset using the [Scikit-learn](http://scikit-learn.org/stable/) toolkit again.

The evaluation data and its targets stored in the same format as the training data is loaded.

In [21]:
import numpy as np
# Read testing data and targets
X_test = np.load('data/uwb/X_test_p3.npy')
y_test = np.load('data/uwb/y_test.npy')
print('# examples: {}'.format(X_test.shape[0]))
print('# features: {}'.format(X_test.shape[1]))

# examples: 18026
# features: 32


The prediction of the classifier on the test data:

In [9]:
y_pred = gs.best_estimator_.predict(X_test)

The comparison of the predictive and reference labels can be done in terms of _recall_, _precision_, and _F1-score_.

In [10]:
from sklearn.metrics import precision_score, recall_score, f1_score
print('Recall (R)   : {:.2%}'.format(recall_score(y_test, y_pred)))
print('Precision (P): {:.2%}'.format(precision_score(y_test, y_pred)))
print('F1-score (F1): {:.2%}'.format(f1_score(y_test, y_pred)))

Recall (R)   : 96.46%
Precision (P): 98.09%
F1-score (F1): 97.27%


Note that the results may slightly differ from those presented in the [paper](paper/matousek_is2017_submit.pdf) due to the random element present in the setting of both the extremely randomized trees classifier and the randomized grid search.

### Evaluation on the CMU data

This time, the trained classifier is evaluated on the [CMU](http://festvox.org/dbs/index.html) test datasets. Since hand-crafted GCIs are not available for these datasets, we used the [Multi-Phase Algorithm](http://www.sciencedirect.com/science/article/pii/S0167639311000094) (MPA) to detect GCIs from the contemporaneous electroglottograph (EGG) signal and used the detected GCIs as the reference ones.

Only the first 100 utterances of each voice were used in this example.

Note that the results shown below concern the "classification-way" of evaluating the performance of a classifier and they were not presented in the [paper](paper/matousek_is2017_submit.pdf). Instead, a "standard" comparison of the detected and reference GCIs in terms of other common measures was conducted. Please see the [paper](paper/matousek_is2017_submit.pdf) for more details.

#### ARCTIC BDL voice

The prediction of the classifier on the CMU ARCTIC [BDL](http://festvox.org/cmu_arctic/dbs_bdl.html) test data:

In [19]:
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score

# Read testing data and targets
X_test = np.load('data/cmu/bdl/X_test_p3.npy')
y_test = np.load('data/cmu/bdl/y_test.npy')
print('# testing examples: {}'.format(X_test.shape[0]))
print('# features        : {}'.format(X_test.shape[1]))

# The prediction of the classifier on the CMU BDL test data
y_pred = gs.best_estimator_.predict(X_test)

# The comparison of the predictive and reference labes can be done in terms of recall, precision, and F1-score.
print('')
print('Recall (R)   : {:.2%}'.format(recall_score(y_test, y_pred)))
print('Precision (P): {:.2%}'.format(precision_score(y_test, y_pred)))
print('F1-score (F1): {:.2%}'.format(f1_score(y_test, y_pred)))

# testing examples: 34485
# features        : 32

Recall (R)   : 97.61%
Precision (P): 94.69%
F1-score (F1): 96.13%


#### ARCTIC SLT voice

The prediction of the classifier on the CMU ARCTIC [SLT](http://festvox.org/cmu_arctic/dbs_slt.html) test data:

In [22]:
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score

# Read testing data and targets
X_test = np.load('data/cmu/slt/X_test_p3.npy')
y_test = np.load('data/cmu/slt/y_test.npy')
print('# testing examples: {}'.format(X_test.shape[0]))
print('# features        : {}'.format(X_test.shape[1]))

# The prediction of the classifier on the CMU SLT test data
y_pred = gs.best_estimator_.predict(X_test)

# The comparison of the predictive and reference labes can be done in terms of recall, precision, and F1-score.
print('')
print('Recall (R)   : {:.2%}'.format(recall_score(y_test, y_pred)))
print('Precision (P): {:.2%}'.format(precision_score(y_test, y_pred)))
print('F1-score (F1): {:.2%}'.format(f1_score(y_test, y_pred)))

# testing examples: 42156
# features        : 32

Recall (R)   : 99.69%
Precision (P): 95.07%
F1-score (F1): 97.33%


#### CSTR US KED Timit

The prediction of the classifier on the CSTR US [KED](http://festvox.org/dbs/dbs_kdt.html) Timit test data:

In [24]:
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score

# Read testing data and targets
X_test = np.load('data/cmu/ked/X_test_p3.npy')
y_test = np.load('data/cmu/ked/y_test.npy')
print('# testing examples: {}'.format(X_test.shape[0]))
print('# features        : {}'.format(X_test.shape[1]))

# The prediction of the classifier on the CMU SLT test data
y_pred = gs.best_estimator_.predict(X_test)

# The comparison of the predictive and reference labes can be done in terms of recall, precision, and F1-score.
print('')
print('Recall (R)   : {:.2%}'.format(recall_score(y_test, y_pred)))
print('Precision (P): {:.2%}'.format(precision_score(y_test, y_pred)))
print('F1-score (F1): {:.2%}'.format(f1_score(y_test, y_pred)))

# testing examples: 29177
# features        : 32

Recall (R)   : 96.96%
Precision (P): 92.73%
F1-score (F1): 94.79%
